In [1]:
!pip install --user pycocotools

In [1]:
import csv
from shutil import copyfile
from pycocotools.coco import COCO
from tqdm import tqdm
from torchvision import models
import torch.nn as nn
import numpy as np
import os
import sys
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt


In [3]:
#make directory and get annotations for training and testing
!mkdir data
!wget http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip -P ./data/
!unzip ./data/captions_train-val2014.zip -d ./data/
!rm ./data/captions_train-val2014.zip

mkdir: cannot create directory ‘data’: File exists
--2020-02-22 01:20:57--  http://msvocds.blob.core.windows.net/annotations-1-0-3/captions_train-val2014.zip
Resolving web.ucsd.edu (web.ucsd.edu)... 132.239.1.230, 132.239.1.231
Connecting to web.ucsd.edu (web.ucsd.edu)|132.239.1.230|:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 19673183 (19M) [application/octet-stream Charset=UTF-8]
Saving to: ‘./data/captions_train-val2014.zip’

captions_train-val2 100%[===================>]  18.76M  --.-KB/s    in 0.08s   

2020-02-22 01:20:57 (224 MB/s) - ‘./data/captions_train-val2014.zip’ saved [19673183/19673183]

Archive:  ./data/captions_train-val2014.zip
replace ./data/annotations/captions_train2014.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [2]:
coco = COCO('./data/annotations/captions_train2014.json')

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!


In [3]:
#get ids of training images
with open('TrainImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    trainIds = list(reader)
    
trainIds = [int(i) for i in trainIds[0]]

In [4]:
for img_id in trainIds:
    path = coco.loadImgs(img_id)[0]['file_name']
    copyfile('/datasets/COCO-2015/train2014/'+path, './data/images/train/'+path)

KeyboardInterrupt: 

In [2]:
cocoTest = COCO('./data/annotations/captions_val2014.json')

loading annotations into memory...
Done (t=0.31s)
creating index...
index created!


In [3]:
with open('TestImageIds.csv', 'r') as f:
    reader = csv.reader(f)
    testIds = list(reader)
    
testIds = [int(i) for i in testIds[0]]

In [4]:
for img_id in testIds:
    path = cocoTest.loadImgs(img_id)[0]['file_name']
    copyfile('/datasets/COCO-2015/val2014/'+path, './data/images/test/'+path)

In [5]:
coco.imgToAnns[302443]

[{'image_id': 302443,
  'id': 411,
  'caption': 'Several metal balls sit in the sand near a group of people.'},
 {'image_id': 302443,
  'id': 1419,
  'caption': 'People standing around many silver round balls on the ground.'},
 {'image_id': 302443,
  'id': 2547,
  'caption': 'Silver balls are lined up in the sand as people mill about in the background.'},
 {'image_id': 302443,
  'id': 2574,
  'caption': 'Silver balls on sand with people walking around. '},
 {'image_id': 302443,
  'id': 3309,
  'caption': 'silver balls laying on the ground around a smaller red ball. '}]

In [6]:
seqs=[]
for img_id in trainIds:
    for i in range(0,len(coco.imgToAnns[img_id])):
        seqs.append(coco.imgToAnns[img_id][i]['caption'])

In [26]:
seqs

['A group of friends sitting on a bench near the water. ',
 'A group of men sitting on  a bench near the water',
 'a couple of people sit on a wooden bench ',
 'A group of people are sitting on a bench looking at the water.',
 'Several men sitting on a bench by a lake.',
 'A tall giraffe standing on a  lush green field.',
 'a large giraffe that is in some green grass',
 'A giraffes stands on bright green grass against a pale blue sky.',
 'a giraffe standing on a grassy hill with its head turned looking backwards',
 'A giraffe standing alone in a field looking back.',
 'A group of peacocks trapped in a pen.',
 'some blue yellow and white peacocks are in their pen',
 'a group of peacocks standing by a fence ',
 'Two colorful blue and green feathered peacocks in a cage together. ',
 'Two peacocks are standing in a fenced enclosure.',
 'A man eats a large piece of pizza.',
 'A man holding a slice of pizza up to his mouth.',
 'A man is taking a bite out of a pizza. ',
 'A man that is holdin

In [7]:
vocab = ['<pad>', '<start>', '<end>', '<unk>'] + sorted(set([word for seq in seqs for word in seq.split()]))

In [24]:
import re
for i,line in enumerate(vocab):
     vocab[i] = line.lower()

In [25]:
vocab

['<pad>',
 '<start>',
 '<end>',
 '<unk>',
 '!',
 '"',
 '".',
 '"....hammertime!"',
 '"20',
 '"24"',
 '"3"',
 '"30".',
 '"350"',
 '"350",',
 '"4-way"',
 '"@me"',
 '"a',
 '"air."',
 '"airport',
 '"albani"',
 '"as',
 '"bohemian',
 '"bicycles',
 '"bill',
 '"bliss',
 '"bourbon"',
 '"broadway"',
 '"broadway".',
 '"curly',
 '"canada"',
 '"cardiff',
 '"cesar\'s',
 '"charlottesville"',
 '"citi',
 '"coca',
 '"congratulations',
 '"continuum"',
 '"coronation',
 '"czech',
 '"dad"',
 '"divided',
 '"dixie',
 '"do',
 '"dyrgas',
 '"end"',
 '"eason"',
 '"end',
 '"eva',
 '"flinders',
 '"farm',
 '"fire',
 '"first',
 '"fitzgerald"',
 '"free',
 '"frisco',
 '"gentlemen".',
 '"garden',
 '"gay',
 '"give',
 '"greenwave"',
 '"hamma\'',
 '"harry',
 '"harts',
 '"homophobia"',
 '"i',
 '"it',
 '"just',
 '"jenny"',
 '"karmill',
 '"kingfisher"',
 '"leroy"',
 '"la',
 '"le',
 '"leroy"',
 '"live',
 '"london',
 '"los',
 '"love',
 '"lunchtime',
 '"monday',
 '"make',
 '"marketplace',
 '"massachusetts',
 '"meow".',
 '"mexica

In [58]:
vectorized_seqs = [[vocab.index(tok) for tok in seq.split()]for seq in seqs]
vectorized_seqs

[[322, 10512, 13575, 9874, 16966, 13628, 4172, 5414, 13282, 18713, 20038],
 [322, 10512, 13575, 12749, 16966, 13628, 4172, 5414, 13282, 18713, 20035],
 [4172, 7565, 13575, 14353, 16954, 13628, 4172, 20422, 5414],
 [322,
  10512,
  13575,
  14353,
  4655,
  16966,
  13628,
  4172,
  5414,
  12322,
  4790,
  18713,
  20038],
 [3387, 12749, 16966, 13628, 4172, 5414, 6209, 4172, 11850],
 [322, 18473, 10164, 17746, 13628, 4172, 12394, 10434, 9373],
 [4172, 11921, 10164, 18708, 11467, 11281, 17396, 10434, 10382],
 [322,
  10170,
  17750,
  13628,
  5942,
  10434,
  10382,
  4331,
  4172,
  13968,
  5657,
  17090],
 [4172,
  10164,
  17746,
  13628,
  4172,
  10400,
  10928,
  20380,
  11499,
  10768,
  19378,
  12322,
  4966],
 [322, 10164, 17746, 4448, 11281, 4172, 9371, 12322, 4930],
 [322, 10512, 13575, 14256, 19184, 11281, 4172, 14328],
 [17396, 5657, 20541, 4513, 20211, 14256, 4655, 11281, 18719, 14326],
 [4172, 10512, 13575, 14256, 17746, 6209, 4172, 9332],
 [3880, 7157, 5657, 4513, 10

In [67]:
with open("vectorized_dict.txt", "w") as txt_file:
    for line in vectorized_seqs:
        txt_file.write((str(line)) + "\n")

In [31]:
resnet_model = models.resnet50(pretrained=True)
print(resnet_model)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/xdg-cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 104MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [32]:
class resnet_encoder(nn.Module):
    def __init__(self, num_classes, fine_tuning=False):
        super(resnet_encoder, self).__init__()
        for param in resnet_model.parameters():
            params.requires_grad = fine_tuning
        self.conv1 = resnet_model.conv1
        self.bn1 = resnet_model.bn1
        self.relu = resnet_model.relu
        self.maxpool = resnet_model.maxpool
        self.layer1 = resnet_model.layer1
        self.layer2 = resnet_model.layer2
        self.layer3 = resnet_model.layer3
        self.layer4 = resnet_model.layer4
        self.avgpool = resnet_model.avgpool
        self.last = nn.Linear(2048, num_classes)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = self.last(out)
        
        return out
        

In [33]:
class resnet_deocder(nn.Module):

    def __init__(self, input_dim, hidden_dim, num_layers, output_size):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.input2hidden = nn.Linear(input_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers)
        self.hidden2tag = nn.Linear(hidden_dim, output_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        h = torch.zeros(self.num_layers,1,self.hidden_dim)
        c = torch.zeros(self.num_layers,1,self.hidden_dim)
        return (Variable(h.cuda()), Variable(c.cuda()))

    def forward(self, x):
        x_len = len(x)
        x = self.input2hidden(x)
        x, self.hidden = self.lstm(x.view(x_len, 1, -1), self.hidden)
        x = self.hidden2tag(x.view(x_len, -1))
        return x